<a href="https://colab.research.google.com/github/Petrichoeur/TER/blob/master/MultiStream/process_list_for_json_with_txt_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd  
import numpy as np
from time import gmtime, strftime

**Remarques:** 
* Le nom de l'enfant est à modifier à deux endroits.
* Il faut remplacer la liste des dates par celle correspondante a l'enfant.

In [2]:
#Chargement du jeu de donnees
data_child = pd.read_csv('/Users/marine/Desktop/Projet_TER/TER/data/data_theophile_final.csv',
                           sep = '\t',
                           encoding = 'utf-8',
                           index_col = False)

# Supression de la colonne qui duplique l'index
#data_child = data_child.drop(columns = 'Unnamed: 0') 
# on a pas besoin car vu qu'apres on prend que deux colonnes cela l'enleve aussi
# Mettre dans l'ordre les valeurs (verification car normalement c'est deja le cas)
#data_child = data_child.sort_values(by = ['age','seconde_debut'])
# Creer un indexage a partir de cet ordre
#data_child['ind']=[ i for i in range(len(data_child))]
#data_child= data_child.set_index('ind')

#data_child.head()

In [3]:
# Recuperation de la partie phonetique des intervention de l'enfant
data_child = data_child.where(data_child['type']=='pho').dropna(subset=['type'])
# Conservation des deux colonnes avec des informations pertinantes 
data_child = data_child[['age','contenu']]

#data_child.head()

In [4]:
data_child.age.unique()

array(['1_00_09', '1_01_14', '1_02_10', '1_03_02', '1_04_01', '1_05_05',
       '1_06_08', '1_07_01', '1_08_08', '1_09_06', '1_10_00', '1_11_07',
       '1_11_25', '2_00_20', '2_02_08', '2_03_01', '2_04_05', '2_04_29',
       '2_06_03', '2_07_04', '2_07_28', '2_09_07', '2_10_01', '2_10_28',
       '2_11_28', '3_02_00', '3_03_02', '3_04_10', '3_05_11', '3_06_10',
       '3_07_08', '3_08_15', '3_10_00', '3_11_15', '4_01_24', '4_03_29',
       '4_06_20', '4_09_07', '4_11_11'], dtype=object)

In [5]:
data_child.age.unique().shape

(39,)

In [6]:
age = data_child['age']

l=[i for i in range(len(age))]
k=[age.values]
k.append(l)
tuples = list(zip(*k))

index = pd.MultiIndex.from_tuples(tuples, names=['age', 'index'])
data_child.set_index(index,inplace=True)

#data_child.head()

In [7]:
data_child=data_child[['contenu']]

#data_child.head()

In [8]:
data_child['contenu']=data_child['contenu'].apply(lambda x: str(x))
#data_child.head()

In [9]:
mots = {}
for i in list(age.values):
    # recupere toutes les phrases a la i eme date
    phrases = data_child['contenu'].loc[i].values
    # creer une variable texte avec toutes les phrases
    phrase = ' '.join(phrases)
    # sépare en mot
    mots[i] = phrase.split()
    
#se parcourt avec un age en parametre
# D["1_00_05"]

In [10]:
df = {} 
age_unique = age.unique()

for i in age_unique : 
    df[i]=pd.DataFrame(mots[i], index= [i for _ in range(len(mots[i]))] , columns=['mot']) 

df = pd.concat([df[i] for i in age_unique]) 

#df.head()

In [11]:
df['index_age']=df.index 
forjson=df[['index_age','mot']]

# forjson.head()

In [12]:
# Creation du dicctionnaire des phonemes 
IPA = "a,r,l,e,s,i,ɛ,ə,t,k,p,d,m,ɑ̃,n,u,v,o,y,ɔ̃,ʒ,ɔ,ɛ̃,f,b,j,w,ɥ,z,ø,ʃ,œ̃,œ,g,ɑ,ɲ,tr,kr,dr,gr" 
IPA2 = IPA.split(',')
Dic = {'0':'ɑ̃','1':'ɔ̃','2':'ɛ̃','3':'œ̃','4':'tr','5':'kr','6':'dr','7':'gr'}
IPA_ = []
def ipa_(x):
  x=x.replace('ʁ','r') 
  x=x.replace('ɑ̃','0')
  x=x.replace('ɔ̃','1') 
  x=x.replace('ɛ̃', '2') 
  x=x.replace('œ̃','3') 
  x=x.replace('tr','4')
  x=x.replace('kr','5')
  x=x.replace('dr','6')
  x=x.replace('gr','7')
  return x 

In [13]:
# Fonction pour diviser le contenu
def splitage(x): 
    l=[]
    for y in x: 
        l.append(y)
    return l

In [14]:
# Transforme les phonemes composee de deux caracteres en un
forjson['mot']=forjson['mot'].apply(lambda x: ipa_(str(x)))
# Decoupage des mot en phonemes
forjson['mot_ut']=forjson['mot'].apply(lambda x: splitage(str(x))) 

# forjson.head()

In [15]:
D={}
for i in age_unique:
    D[i]=forjson[forjson['index_age']== i]


name_for_pho={}
for i in age_unique: 
    l=D[i].mot.values
    k=list(D[i].mot_ut.values)
    name=[] 
    for p in range(len(k)):
        for j in range(len(k[p])):
            name.append(l[p])  
        name_for_pho[i]=name

final={} 
for i in age_unique:
    final[i]=pd.DataFrame([name_for_pho[i],D[i]['mot_ut'].sum()],index=['mot','phoneme']).transpose()

In [16]:
def discrimi(x):
    if x not in ipa_(IPA): 
        return np.nan
    else :
        return x 
    
for i in final.keys():
    final[i]['phoneme'] = final[i]['phoneme'].apply(lambda x: discrimi(x) ) 
    final[i].dropna(inplace=True)

In [17]:
dates = ['09/01/2001',
'14/02/2001',
'13/03/2001',
'04/04/2001',
'04/05/2001',
'07/06/2001',
'11/07/2001',
'04/08/2001',
'08/09/2001',
'07/10/2001',
'31/10/2001',
'08/12/2001',
'26/12/2001',
'20/01/2002',
'11/03/2002',
'03/04/2002',
'08/05/2002',
'01/06/2002',
'06/07/2002',
'07/08/2002',
'31/08/2002',
'09/10/2002',
'02/11/2002',
'29/11/2002',
'30/12/2002',
'04/03/2003',
'05/04/2003',
'14/05/2003',
'14/06/2003',
'14/07/2003',
'12/08/2003',
'16/09/2003',
'01/11/2003',
'17/12/2003',
'25/02/2004',
'31/03/2004',
'01/05/2004',
'23/07/2004',
'08/10/2004',
'12/12/2004',
'16/05/2005',
'27/10/2005',
'01/05/2006'
]
#dates

In [18]:
len(dates)

43

In [19]:
fichier = open("TheophileDataJson.txt", "a")
itera=0
for key,df_val in final.items(): 
    for i in range(len(df_val)):
        to_write=strftime('{"date_time": "'+dates[itera]+' %H:%M:%S","text": "'+df_val.mot.iloc[i]+'","value": "'+df_val.phoneme.iloc[i]+'"}, \n', gmtime(i))
        fichier.write(to_write)
    itera+=1


fichier.close()
# itera correspond a la i eme date
#key = nom du fichier
#df_val = le contenu ; mot et pho
#len(df_val) nous renseigne sur le nbr d'itteration par date